In [88]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from collections import Counter
from os.path import join as oj

import numpy as np
from matplotlib import pyplot as plt
plt.style.use('dark_background')
import pandas as pd
import pickle as pkl
from tqdm import tqdm
from viz import *

# out_dir = 'results/aug7_1' # combined the different abdTender values
# out_dir = 'results/aug6_6/' # has a lot of results
out_dir = 'results/aug7_logistics' # has hyperparams for logistic

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# compare all models

In [89]:
def load_results(out_dir):
    r = []
    for fname in tqdm(os.listdir(out_dir)):
        d = pkl.load(open(oj(out_dir, fname), 'rb'))
        d['param_name'] = fname[:-4]
        r.append(pd.Series(d))
    r = pd.concat(r, axis=1, sort=False).T.infer_objects()
    r = r.reindex(sorted(r.columns, reverse=True), axis=1) # sort the column names
    r = r.round(3)
    r = r.set_index('param_name')
    return r

results = load_results(out_dir)
# print(results.keys())

100%|██████████| 6/6 [00:00<00:00, 352.81it/s]


## look at prediction metrics

Most important thing is to have very low false negatives (very close to zero). After that, we also want to minimize the false positives.

In [90]:
# prev rule: sensitivity 0.97, specificity 0.425
r = results
# print(r.keys())
# r = r[[k for k in r if not 'std' in k]]
# r = r[[k for k in r if not '_test' in k]]
# r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
# r = r.sort_values(by=['balanced_accuracy'], ascending=False)
# r = r[r.model_type == 'grl']
# r = r[r.model_type == 'brl']
r = r[r['sensitivity_cv'] > 0.8] # what fraction of true pos do we return?
r = r[r['specificity_cv'] > 0.4] # what fraction of true negs do we return?

r = r.sort_values(by=['sensitivity_cv', 'specificity_cv'], ascending=False)
r = r[['sensitivity_cv', 'specificity_cv', # 'precision',
       'sensitivity_test1', 'specificity_test1',
       'sensitivity_test2', 'specificity_test2',
       'fn_cv', 'tn_cv', 'fp_cv', 'tp_cv',
       'precision_cv', 'roc_auc_cv', 
#        'f1_cv', 'balanced_accuracy_cv', 'accuracy_cv',
       'fn_test1', 'tn_test1', 'fp_test1', 'tp_test1',
       'fn_test2', 'tn_test2', 'fp_test2', 'tp_test2']]
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
# r = r[r.index == 'logistic_select_lasso=27_ros=100']
# r.style.background_gradient(cmap='viridis', axis=0) # columns differently colored
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
r

,sensitivity_cv,specificity_cv,sensitivity_test1,specificity_test1,sensitivity_test2,specificity_test2,fn_cv,tn_cv,fp_cv,tp_cv,precision_cv,roc_auc_cv,fn_test1,tn_test1,fp_test1,tp_test1,fn_test2,tn_test2,fp_test2,tp_test2
param_name,,,,,,,,,,,,,,,,,,,,


**look at feat importances**

In [ ]:
best_model = 'logistic_select_stab_lasso=5_sample_weights=750' #logistic_select_rf=5_smote=5'
r = results[results.index.str.contains(best_model)].iloc[0]

d = pd.DataFrame.from_dict({
    'feats': r['feat_names_selected'],
    'imps': r['imps_best'],
})
d = d.iloc[np.argsort(d['imps'])[::-1]]
d = d.round(2)
d.fillna(0).style.background_gradient(cmap=cm_rev) #, axis=0) #, subset=subset) # rows differently colored

In [77]:
r[['sensitivity_cv', 'specificity_cv', # 'precision',
       'sensitivity_test1', 'specificity_test1',
       'sensitivity_test2', 'specificity_test2']]

sensitivity_cv       0.969
specificity_cv       0.521
sensitivity_test1    0.877
specificity_test1    0.513
sensitivity_test2    0.984
specificity_test2    0.515
Name: logistic_select_stab_lasso=5_sample_weights=750, dtype: object